# Max Voting

In [180]:
import matplotlib.pyplot as plt
import pandas as pd
from xgboost import XGBClassifier
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier

#### Read data

In [181]:
df = pd.read_csv('../../../datasets/parte2/treino/dataset_prepared.csv', na_filter=False)

In [182]:
replace_map = {'None':0, 'Low':1, 'Medium':2, 'High':3, 'Very High':4}

df['injection'] = df['injection'].replace(replace_map).astype(int)

#### X and y arrays

In [183]:
X = df.drop(['injection'], axis=1)
y = df[['injection']]

#### Train Test Split

Now let's split the data into a training set and a testing set. We will train out model on the training set and then use the test set to evaluate the model.

In [184]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023, stratify=y)

## XGBoost

#### Training 

Using GridSearchCV to find the best hyperparameters

In [185]:
xgb_model = XGBClassifier(random_state=2023, learning_rate=0.1, max_depth=4, n_estimators=200)

In [186]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

## Neural Networks

### Normalization

In [187]:
scaler_X = MinMaxScaler(feature_range=(0, 1)).fit(X)
X_train = pd.DataFrame(scaler_X.transform(X_train[X_train.columns]), columns=X_train.columns)

#y = pd.DataFrame(scaler_y.transform(y[y.columns]), columns=y.columns)

X_test = pd.DataFrame(scaler_X.transform(X_test[X_test.columns]), columns=X_test.columns)

#### Train Test Split

Now let's split the data into a training set and a testing set. We will train out model on the training set and then use the test set to evaluate the model.

In [188]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [189]:
def build_model(activation='relu', learning_rate=0.005, dropout_rate=0.1):
    ann_model = Sequential()
    ann_model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))
    ann_model.add(Dropout(dropout_rate))  # Adiciona Dropout após a primeira camada densa
    ann_model.add(Dense(32, activation=activation))
    ann_model.add(Dropout(dropout_rate))  # Adiciona Dropout após a segunda camada densa
    ann_model.add(Dense(5, activation='softmax')) # output 
    
    #Compile the model
    ann_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy'])

    return ann_model 

In [190]:
ann_model = build_model()
ann_model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 64)                1408      
                                                                 
 dropout_40 (Dropout)        (None, 64)                0         
                                                                 
 dense_61 (Dense)            (None, 32)                2080      
                                                                 
 dropout_41 (Dropout)        (None, 32)                0         
                                                                 
 dense_62 (Dense)            (None, 5)                 165       
                                                                 
Total params: 3,653
Trainable params: 3,653
Non-trainable params: 0
_________________________________________________________________


In [191]:
ann_model = KerasClassifier(build_fn=build_model, batch_size=32, validation_split=0.2, epochs=100)

In [192]:
ann_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100


/home/gabs/.conda/envs/daa1/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


259/259 [==============================] - 1s 4ms/step - loss: 0.8021 - accuracy: 0.7274 - val_loss: 0.5713 - val_accuracy: 0.7683
Epoch 2/100
259/259 [==============================] - 1s 4ms/step - loss: 0.5331 - accuracy: 0.7866 - val_loss: 0.4668 - val_accuracy: 0.8057
Epoch 3/100
259/259 [==============================] - 1s 5ms/step - loss: 0.4778 - accuracy: 0.8023 - val_loss: 0.4359 - val_accuracy: 0.8174
Epoch 4/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4565 - accuracy: 0.8088 - val_loss: 0.4342 - val_accuracy: 0.8199
Epoch 5/100
259/259 [==============================] - 1s 5ms/step - loss: 0.4428 - accuracy: 0.8142 - val_loss: 0.4126 - val_accuracy: 0.8268
Epoch 6/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4292 - accuracy: 0.8160 - val_loss: 0.4015 - val_accuracy: 0.8315
Epoch 7/100
259/259 [==============================] - 1s 4ms/step - loss: 0.4205 - accuracy: 0.8204 - val_loss: 0.3925 - val_accuracy: 0.8330
Epoch 8/100

KerasClassifier(
	model=None
	build_fn=<function build_model at 0x7f0ae46704c0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

## Max Voting

In [193]:
hvt_model = VotingClassifier(estimators=[("ann", ann_model), ("xgb", xgb_model)], voting='hard', weights=[1,2])

In [194]:
hvt_model.fit(X_train, y_train)

NotImplementedError: Multilabel and multi-output classification is not supported.

In [ ]:
hvt_score = hvt_model.score(X_test, y_test)
print("Accuracy: %.2f%%" % (hvt_score*100))

In [ ]:
hvt_predictions = hvt_model.predict(X_test)

In [ ]:
print(classification_report(y_test, hvt_predictions))

In [ ]:
hvt_predictions.to_csv('../../../datasets/parte2/teste/ensemble_results.csv', index=False)

In [ ]:
#for model, label in zip([ann_model, xgb_model, hvt_model], ['ann', 'xgb', 'ensemble']):
#    hvt_score = cross_val_score(model, X_test, y_test, scoring='accuracy', cv=10)
#    print("Accuracy: %0.2f (+/- %0.2f) [%s]")